In [1]:
# IMPORT LIBRAIRIES
import numpy as np
import pandas as pd
from tqdm import tqdm
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from python_module.pricing_model import SABRModel
from scipy.optimize import least_squares, minimize

In [2]:
# DEFINE LOCAL VOL & MONTECARLO FUNCTION

def compute_local_volatility(F, K, T, alpha, beta, rho, nu, r, dK=0.001, dT=0.001):
    C     = SABRModel.compute_option(F=F, K=K, T=T, alpha=alpha, beta=beta, rho=rho, nu=nu, r=r, option_type='call')['price'] * np.exp(r*T)
    C_Kp  = SABRModel.compute_option(F=F, K=K+dK, T=T, alpha=alpha, beta=beta, rho=rho, nu=nu, r=r, option_type='call')['price'] * np.exp(r*T)
    C_Km  = SABRModel.compute_option(F=F, K=K-dK, T=T, alpha=alpha, beta=beta, rho=rho, nu=nu, r=r, option_type='call')['price'] * np.exp(r*T) 
    C_Tp  = SABRModel.compute_option(F=F, K=K, T=T+dT, alpha=alpha, beta=beta, rho=rho, nu=nu, r=r, option_type='call')['price'] * np.exp(r*(T+dT))
    C_Tm  = SABRModel.compute_option(F=F, K=K, T=T-dT, alpha=alpha, beta=beta, rho=rho, nu=nu, r=r, option_type='call')['price'] * np.exp(r*(T-dT))    
    d2C_dK2 = (C_Kp - 2*C + C_Km) / (dK**2)
    dC_dT   = (C_Tp - C_Tm) / (2*dT)
    if (K**2 * d2C_dK2) != 0:
        var = 2 * dC_dT / (K**2 * d2C_dK2)
        if var >= 0:
            lv   =  np.sqrt(var)
        else:
            lv = 0    
    else:
        lv = 0
    return lv

def simulate_sabr_mc(F0, vol0, beta, nu, rho, T, N, n_paths, seed=None):
    if seed is not None:
        np.random.seed(seed)

    dt = T / N
    sqrt_dt = np.sqrt(dt)
    times = np.linspace(0, T, N+1)

    # Pre‐allocate
    F_paths = np.zeros((n_paths, N+1))
    sigma_paths = np.zeros((n_paths, N+1))

    # Set initial values
    F_paths[:, 0] = F0
    sigma_paths[:, 0] = vol0

    for i in range(N):
        # generate two independent normals
        z1 = np.random.standard_normal(n_paths)
        z2 = np.random.standard_normal(n_paths)
        # correlated increments
        dW = z1 * sqrt_dt
        dZ = (rho * z1 + np.sqrt(1.0 - rho**2) * z2) * sqrt_dt

        # log‐Euler update for sigma to keep it positive
        sigma_paths[:, i+1] = sigma_paths[:, i] * np.exp(-0.5 * nu**2 * dt + nu * dZ)

        # Euler update for F
        F_paths[:, i+1] = (
            F_paths[:, i]
            + sigma_paths[:, i] * (F_paths[:, i]**beta) * dW
        )

    return times, F_paths, sigma_paths

In [3]:
# INPUTS
F0 = 100
beta = 1
nu = 0.5
rho = -0.8 
alpha = 0.2
r = 0
n_steps = 252
T = n_steps/252
n_paths = 1

In [4]:
# SIMULATE MONTACARLO PATH

times, F_paths, sigma_paths = simulate_sabr_mc(F0, alpha, beta, nu, rho, T, n_steps, n_paths, seed=None)
forward_price = pd.Series(F_paths[0, :])
vol_process = pd.Series(sigma_paths[0, :])

In [5]:
# GENERATE LOCAL VOL

local_vol_dict = dict()
for K in np.linspace(start=95, stop=105, num=50): #range(95, 105):
    for T in range(10, n_steps, 10):
        local_vol_dict[(K, T)] = compute_local_volatility(F=F0, K=K, T=T/252, alpha=alpha, beta=1, rho=rho, nu=nu, r=r)
local_vol_df = pd.Series(local_vol_dict).reset_index()
local_vol_df.columns = ['K', 'T', 'LV']

In [6]:
# PLOT LOCAL VOL

pivot = local_vol_df.pivot(index='T', columns='K', values='LV')

K_vals = pivot.columns.values
T_vals = pivot.index.values     
LV_vals = pivot.values

fig = go.Figure(data=[go.Surface(x=K_vals, y=T_vals, z=LV_vals, colorscale='Viridis', showscale=True)])

fig.update_layout(title="Local Volatility Surface",scene=dict(xaxis_title='Strike (K)', yaxis_title='Time to Expiry (T)', zaxis_title='Local Volatility')
                  ,width=1300, height=800)
fig.show()

px.line(np.round(local_vol_df[local_vol_df["T"]==10].set_index('K')['LV']*100,6), title="Local Volatility Surface @ Time To Expiry = 10d" ,width=1300, height=800)



In [7]:
# COMPUTE TIMESERIE LOCAL VOL
local_vol_list = list()
for index in forward_price.index:
    if index == 0:
        local_vol_list.append(np.nan)
    else:
        T = index/252
        K = forward_price.loc[index] 
        lv = compute_local_volatility(F=F0, K=K, T=T, alpha=alpha, beta=beta, rho=rho, nu=nu, r=r)
        local_vol_list.append(lv)

In [8]:
# PLOT TIMESERIES: FORWARD PRICE, ALPHA, LOCAL VOL
df = pd.DataFrame()
df['forward_price'] = forward_price
df['vol_process'] = vol_process
df['local_vol'] = local_vol_list

x = df.index
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=x, y=df['forward_price'], name='Forward Process'), secondary_y=False)
fig.add_trace(go.Scatter(x=x, y=df['vol_process'], name='Vol Process'), secondary_y=True)
fig.add_trace(go.Scatter(x=x, y=df['local_vol'], name='Local Vol'), secondary_y=True)
fig.update_layout(title="Forward vs Vol Processes", xaxis_title="Time", legend=dict(orientation="h", yanchor="bottom", y=1.02))
fig.update_yaxes(title_text="Forward Process", secondary_y=False)
fig.update_yaxes(title_text="Volatility",      secondary_y=True)
fig.show()

In [9]:
# GRID SEARCH: PARAMETERS
F0 = forward_price.iloc[0]
results = dict()
for alpha_ in [alpha]: #tqdm(np.linspace(start=0.01, stop=1, num=5)):
    for rho_ in tqdm(np.linspace(start=-0.99, stop=0.99, num=5)):
        for nu_ in np.linspace(start=0.1, stop=0.9, num=9):
            local_vol_list = list()
            for index in forward_price.index:
                if index == 0:
                    local_vol_list.append(np.nan)
                else:
                    T = index/252
                    K = forward_price.loc[index] 
                    lv = compute_local_volatility(F=F0, K=K, T=T, alpha=alpha_, beta=beta, rho=rho_, nu=nu_, r=0)
                    local_vol_list.append(lv)
            df = pd.DataFrame()
            df['forward_price'] = forward_price
            df['local_vol'] = local_vol_list
            df['realized_vol'] = np.sqrt(np.log(df['forward_price']).diff().pow(2) * 252).shift(-1)
            df = df.dropna()
            mean_vol_error = (df['local_vol']-df['realized_vol']).abs().mean()
            mean_var_error = (df['local_vol'].pow(2)-df['realized_vol'].pow(2)).abs().mean()
            results[(alpha_, rho_, nu_)] = mean_var_error

100%|██████████| 5/5 [00:11<00:00,  2.21s/it]


In [10]:
print(pd.Series(results).sort_values())

0.2  -0.990  0.8    0.014442
             0.9    0.014480
             0.7    0.014628
             0.6    0.015201
             0.5    0.016233
             0.4    0.017798
             0.3    0.019800
     -0.495  0.9    0.021131
             0.8    0.021223
             0.7    0.021495
             0.6    0.021940
     -0.990  0.2    0.022314
     -0.495  0.5    0.022578
             0.4    0.023417
             0.3    0.024484
     -0.990  0.1    0.025493
     -0.495  0.2    0.025815
             0.1    0.027379
      0.000  0.1    0.029328
             0.2    0.029688
             0.3    0.030296
             0.4    0.031179
      0.495  0.1    0.031340
      0.000  0.5    0.032355
      0.990  0.1    0.033438
      0.495  0.2    0.033845
      0.000  0.6    0.033855
             0.7    0.035706
      0.495  0.3    0.036733
      0.000  0.8    0.037999
      0.990  0.2    0.038228
      0.495  0.4    0.040102
      0.000  0.9    0.040706
      0.990  0.3    0.043569
      0.495  0